In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_vertical_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,4959.09,0.11,0.03,0.03,0.00,0.00,-0.00,-0.00
3,0.11,1249.25,-0.01,0.01,-0.00,0.00,-0.00,0.00
4,0.03,-0.01,275.89,0.00,0.00,-0.00,0.00,-0.00
5,0.03,0.01,0.00,55.95,0.00,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,9.98,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,1.74,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.32,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.07


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00005,0.00000,0.00000,-0.00000,-0.00002
3,0.00004,1.00000,-0.00003,0.00004,-0.00001,0.00002,-0.00005,0.00009
4,0.00003,-0.00003,1.00000,0.00002,0.00001,-0.00006,0.00007,-0.00006
5,0.00005,0.00004,0.00002,1.00000,0.00002,0.00002,0.00002,-0.00004
6,0.00000,-0.00001,0.00001,0.00002,1.00000,0.00002,0.00003,0.00004
7,0.00000,0.00002,-0.00006,0.00002,0.00002,1.00000,-0.00004,0.00001
8,-0.00000,-0.00005,0.00007,0.00002,0.00003,-0.00004,1.00000,0.00003
9,-0.00002,0.00009,-0.00006,-0.00004,0.00004,0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.16497616380285557

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.07848369e+06 1.91121563e+05 9.06357629e+03 3.57887235e+02
 1.06904296e+01 2.88546678e-01 7.69982153e-03 2.02869351e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999234,-0.038291,-0.007870,-0.001587,-0.000284,-0.000049,-0.000009,-0.000002
1,0.037944,0.998492,-0.038949,-0.007456,-0.001329,-0.000230,-0.000044,-0.000009
2,0.009259,0.038262,0.998293,-0.042510,-0.007221,-0.001259,-0.000230,-0.000052
3,0.002243,0.008933,0.041788,0.997981,-0.046264,-0.007732,-0.001432,-0.000313
4,0.000499,0.001983,0.008984,0.045348,0.997075,-0.059846,-0.010670,-0.002316
5,0.000114,0.000454,0.002064,0.010155,0.058081,0.993481,-0.095519,-0.019746
6,0.000030,0.000121,0.000543,0.002688,0.015080,0.088312,0.972463,-0.215132
7,0.000014,0.000054,0.000249,0.001223,0.006847,0.039405,0.212311,0.976383


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0007656831049732693,
 0.0015082137007812468,
 0.0017067280747976588,
 0.0020187468963792643,
 0.0029253178585244566,
 0.006518665125561762,
 0.02753705631381098,
 0.02361747207630749]